In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import plotly.express as px
import glob
import spacy
import sys
import gc
from IPython.display import display
sys.path.append('../../../../utils')
sys.path.append('..')
import gezi
from gezi import tqdm
from src.util import *
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [ ]:
import sys
sys.path.append('..')
sys.path.append('../../../../utils')
sys.path.append('../../../../third')
import tensorflow as tf
import torch
from absl import flags
FLAGS = flags.FLAGS
from transformers import AutoTokenizer
from datasets import Dataset
from src import config
from src.util import *
import melt as mt
import numpy as np
from gezi import tqdm
import gezi
import husky
import lele

In [ ]:
v = '40'
mns = [
  'mid.electra.sm=start.mui.mis=mid,end.bs=16.y',
  # 'mid.roberta.sm=start.mui.mis=mid,end.bs=16.y',
  # 'mid.xlnet.sm=start.mui.mis=mid,end.bs=16.y',
  # 'mid.deberta-v3.sm=start.mui.mis=end.bs=16.y',
]
root = f'../working/online/{v}/0/'

In [ ]:
model_dirs = [f'{root}/{mn}' for mn in mns]
model_dir = model_dirs[0]
gezi.init_flags()
batch_sizes = [32] * 100
weights = [1] * 100
num_tf_models = len([x for x in model_dirs if 'tf.' in x])
num_tf_models 

In [ ]:
test_inputs = {}
def get_inputs(backbone, mode='test', reset=True, sort=True, double_times=0, split_method=None, multi_inputs=None):
  split_method = split_method if split_method is not None else FLAGS.split_method
  multi_inputs = multi_inputs if multi_inputs is not None else FLAGS.multi_inputs
  if multi_inputs:
    split_methods = [split_method] + FLAGS.multi_inputs_srcs
    res = {}
    for i, split_method in enumerate(split_methods):
      res[i] = get_inputs(backbone, mode, reset, sort, double_times, split_method, multi_inputs=False)
    res.update({
      'id': res[0]['id'],
      'num_words': res[0]['num_words']
    })
    return res
  # TODO 内部之间支持可以减少encode分词调用次数
  multi_inputs = False
  #--- for online infer
  FLAGS.remove_br = False
  if backbone in test_inputs and not reset:
    return test_inputs[backbone]
  else:
    tokenizer = get_tokenizer(backbone)
    files = os.listdir(f'../input/feedback-prize-2021/{mode}')
    TEST_IDS = [f.replace('.txt','') for f in files if 'txt' in f]
    ic(len(TEST_IDS), torch.cuda.is_available(), tokenizer.padding_side)
    if len(TEST_IDS) < 10 and torch.cuda.is_available() and double_times:
      for _ in range(double_times):
        TEST_IDS += TEST_IDS
    ic(len(TEST_IDS))
    MAX_LEN = FLAGS.max_len
    input_ids = np.zeros((len(TEST_IDS), MAX_LEN), dtype='int32')
    attention_mask = np.zeros((len(TEST_IDS), MAX_LEN), dtype='int32')
    word_ids_list = []
    re_pos_list = []
    num_words_list = []
   
    for id_num in tqdm(range(len(TEST_IDS))):
      n = TEST_IDS[id_num]
      name = f'../input/feedback-prize-2021/{mode}/{n}.txt'
      txt = open(name, 'r').read()
      res = encode(txt, tokenizer, split_method=split_method, multi_inputs=multi_inputs) 
      input_ids[id_num,] = res['input_ids']
      attention_mask[id_num,] = res['attention_mask']
     
      word_ids_list.append(res['word_ids'])
      re_pos_list.append(res['relative_positions'])
      num_words_list.append(len(txt.split()))
    
    inputs = {
      'id': TEST_IDS,
      'input_ids': input_ids,
      'attention_mask': attention_mask,
      'word_ids': word_ids_list,
      'relative_positions': re_pos_list,
      'num_words': num_words_list,
    }    
    keys = list(inputs.keys())
    if sort:
      df = pd.DataFrame({k: list(inputs[k]) for k in keys})
      df = df.sort_values('num_words')
      inputs = {
        k: np.asarray(df[k].values) for k in keys
      }
    if not reset:
      test_inputs[backbone] = inputs
    return inputs

In [26]:

ensembler = Ensembler()
for i, model_dir in tqdm(enumerate(model_dirs), total=len(model_dirs)):
  ic(model_dir)
  gezi.restore_configs(model_dir)
  FLAGS.wandb = False
  ic(FLAGS.torch)
  ic(FLAGS.model_dir)
  FLAGS.backbone = '../input/' + FLAGS.backbone.split('/')[-1]
  ic(FLAGS.backbone, FLAGS.max_len)
  model = get_model()
  gezi.load_weights(model, model_dir)
  ic(gezi.get_mem_gb())
  display(pd.read_csv(f'{model_dir}/metrics.csv'))
  inputs = get_inputs(FLAGS.backbone)
 
  # not for roberta-large tf can use 32 bs, torch can use even larger bs, for longformer tf only 8, torch can use 32
  #   batch_size = 32 if FLAGS.torch else 8
  batch_size = 32 
  p = predict(model, inputs, batch_sizes[i], dynamic_keys=['input_ids'], mask_key='attention_mask')
  p.update({
    'id': inputs['id'],
    'word_ids': inputs['word_ids'],
    'num_words': inputs['num_words']
  })
  convert_res(p)
  ensembler.add(p, weights[i])
  if len(inputs['id']) < 1000:
    df = get_preds(p)
    display(df)
    df = link_evidence(df)
    display(df)
    
  del model
  del inputs
  if FLAGS.torch:
    torch.cuda.empty_cache()
  else:
    # only the last tf model should cuda close
    if i + 1 == num_tf_models:
      cuda.select_device(0)
      cuda.close()
  gc.collect()

p = ensembler.finalize()

In [ ]:
df = get_preds(p)
display(df)
df = link_evidence(df)
df

In [ ]:
df.to_csv(f'{model_dir}/submission.csv')